# Assignment: Create AutoGen Chat Agents with User Proxy and Assistant Roles


---

### Objective:
This assignment will guide you through setting up a basic multi-agent conversation using **AutoGen**. You will create two core agent types: `UserProxyAgent` and `AssistantAgent`, understand their distinct roles, and orchestrate a simple automated chat between them to solve a defined problem. This serves as a foundational exercise for building more complex agentic workflows.

---

### Instructions:
1.  **LLM Access**: You will need access to an LLM API. **OpenAI's models (e.g., GPT-4o, GPT-4, GPT-3.5-turbo)** are recommended for their robust function calling capabilities, which AutoGen leverages heavily. You can also use local LLMs via `ollama` or `lmstudio` if configured correctly (see AutoGen docs for details).
2.  **Environment Setup**: Install the necessary Python library: `pip install pyautogen`.
3.  **API Key**: Securely handle your API key. It's best practice to load it from an environment variable or a configuration file (like `OAI_CONFIG_LIST`). For simplicity in this assignment, we'll demonstrate using `os.environ`.
4.  **Jupyter Notebook**: All your code, outputs, observations, and analysis must be documented in this Jupyter Notebook.
5.  **Task Scenario**: The agents will collaborate to solve a simple Python coding problem or a knowledge-based query.
6.  **Analysis**: Explain the roles, interaction flow, and termination conditions.

---

## Part 1: Setup and LLM Configuration
Begin by installing AutoGen and configuring your LLM API key.

### Task 1.1: Install AutoGen
Ensure `pyautogen` is installed in your environment.

In [ ]:
# Install AutoGen (if not already installed)
# !pip install pyautogen --quiet

import autogen
import os

print("AutoGen imported!")

### Task 1.2: Configure LLM
Set up your OpenAI API key (or configuration for other LLMs). For this assignment, we'll assume OpenAI. Replace `YOUR_OPENAI_API_KEY_HERE` with your actual key or load it from an environment variable.

In [ ]:
# --- YOUR OPENAI API KEY HERE ---
# It's highly recommended to load your API key from an environment variable.
# On Linux/macOS: export OPENAI_API_KEY='your_key'
# On Windows (cmd): set OPENAI_API_KEY=your_key

# As an alternative for quick testing, you can uncomment and paste your key directly:
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE"

# Define the LLM configuration list.
# AutoGen will try models in this list in order.
config_list_openai = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o", "gpt-4", "gpt-3.5-turbo"], # Prioritize larger models if available
    },
)

# If OAI_CONFIG_LIST doesn't exist or you prefer direct configuration:
if not config_list_openai:
    # Fallback to direct environment variable if OAI_CONFIG_LIST is not found/configured
    if "OPENAI_API_KEY" in os.environ:
        print("Using OPENAI_API_KEY from environment variable.")
        config_list_openai = [
            {
                "model": "gpt-4o", # or "gpt-4", "gpt-3.5-turbo"
                "api_key": os.environ["OPENAI_API_KEY"],
            }
        ]
    else:
        print("WARNING: OPENAI_API_KEY not found in environment variables or OAI_CONFIG_LIST.")
        print("Please set OPENAI_API_KEY or create OAI_CONFIG_LIST file.")
        config_list_openai = [] # Empty list will cause errors if LLM is needed

print("LLM configuration loaded!")

### Task 1.3: Define the Problem Scenario
Choose a simple problem for your agents to solve. This could be:
* "Write a Python function to check if a string is a palindrome."
* "Explain the concept of 'recursion' in simple terms."
* "Calculate the 10th Fibonacci number using a recursive approach."

For this assignment, let's use the first one as a starting point. Feel free to modify it.

In [ ]:
problem_statement = "Write a Python function named `is_palindrome` that takes a string as input and returns `True` if it's a palindrome (reads the same forwards and backwards, ignoring case and spaces), and `False` otherwise. Provide at least two test cases with expected outputs."

print("Problem statement defined:", problem_statement)

---

## Part 2: Agent Creation
Now, create your `AssistantAgent` and `UserProxyAgent` instances.

### Task 2.1: Create the `AssistantAgent`
Instantiate an `AssistantAgent`. This agent typically represents the AI that generates responses and code.

* **Name**: Choose a descriptive name (e.g., `coder_agent`, `solver`).
* **System Message**: Define its role and objective concisely (e.g., "You are a helpful AI assistant that writes Python code and explains concepts.").
* **LLM Config**: Pass your `config_list_openai`.

In [ ]:
assistant = autogen.AssistantAgent(
    name="PythonCoder",
    system_message="You are a helpful AI assistant that writes clean, functional Python code, provides explanations, and ensures correctness with test cases. Respond with 'TERMINATE' when the task is fully completed and all code and explanations are satisfactory.",
    llm_config={
        "config_list": config_list_openai,
        "temperature": 0.7, # Adjust temperature for creativity/determinism
    },
)

print("AssistantAgent 'PythonCoder' created!")

### Task 2.2: Create the `UserProxyAgent`
Instantiate a `UserProxyAgent`. This agent acts on behalf of the user, can execute code, and often initiates conversations.

* **Name**: Choose a descriptive name (e.g., `user_proxy`, `executor`).
* **Human Input Mode**: Set to `"NEVER"` for fully automated execution (no human input during chat). For debugging or interactive use, `"ALWAYS"` or `"TERMINATE"` can be used.
* **Max Consecutive Auto Reply**: Set a reasonable limit to prevent infinite loops.
* **Code Execution Config**: Enable code execution. This is crucial for AutoGen agents to run generated Python code. Specify a `work_dir` where scripts will be saved and executed.
* **Is Termination Message**: Define a function to determine when the conversation should end. This is how the `UserProxyAgent` knows when to stop relaying messages.

In [ ]:
# Define the termination condition for the UserProxyAgent
def is_termination_message(message):
    return "TERMINATE" in message.get("content", "").upper()

user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER", # Set to NEVER for full automation
    max_consecutive_auto_reply=10, # Max number of turns before stopping
    is_termination_msg=is_termination_message,
    code_execution_config={
        "work_dir": "coding_temp", # Directory to save and execute code
        "use_docker": False, # Set to True if you have Docker installed for isolated execution
    },
)

print("UserProxyAgent 'User' created!")

---

## Part 3: Initiating and Managing the Conversation
Initiate the chat between your two agents and observe their interaction.

### Task 3.1: Start the Chat
Use the `initiate_chat` method of the `UserProxyAgent` to begin the conversation with the `AssistantAgent` on the defined `problem_statement`.

In [ ]:
print("\n--- Initiating the Chat --- ")

# Make sure the coding_temp directory exists
if not os.path.exists("coding_temp"):
    os.makedirs("coding_temp")

chat_result = user_proxy.initiate_chat(
    assistant, # The assistant agent to chat with
    message=problem_statement, # The initial message/problem
    clear_history=True, # Clear previous chat history if any
    silent=False, # Set to True to suppress console output during chat
)

print("\n--- Chat Ended --- ")

### Task 3.2: Observe and Analyze the Interaction
Review the console output from the chat above. AutoGen provides detailed logs of message exchanges and code execution. Pay attention to:

* Which agent is speaking (`User` or `PythonCoder`).
* The content of the messages (code, explanations, execution results).
* When and how code is executed (you'll see `exitcode` and output from the `User` agent).
* The message that ultimately leads to termination (e.g., `TERMINATE`).

---

## Part 4: Analysis and Reflection
Answer the following questions based on your observations of the AutoGen chat.

### Task 4.1: Conversation Review
* **Problem Resolution**: Was the `problem_statement` successfully addressed? Did the `PythonCoder` agent provide correct code and test cases?
* **Agent Contributions**: Describe the key messages and actions taken by the `PythonCoder` (AssistantAgent) and the `User` (UserProxyAgent). How did they collaborate?
* **Code Execution**: Did the `UserProxyAgent` execute code? If so, what was the output of the code execution?
* **Termination**: How did the conversation conclude? Was the `is_termination_msg` function effective in stopping the chat when the assistant issued a 'TERMINATE' message?

### Task 4.2: Role and Functionality Discussion
* **`UserProxyAgent` vs. `AssistantAgent`**: Clearly differentiate the primary responsibilities and capabilities of the `UserProxyAgent` and `AssistantAgent` roles within AutoGen.
* **`human_input_mode`**: Explain the purpose of the `human_input_mode` parameter. What are the implications of setting it to `"NEVER"`, `"ALWAYS"`, or `"TERMINATE"` for different use cases?
* **`is_termination_msg`**: Why is it important to define a clear termination condition for multi-agent conversations? What happens if it's not well-defined?
* **`code_execution_config`**: Discuss the importance of `code_execution_config` for the `UserProxyAgent`. How does it enable agents to test and verify their own solutions?

### Task 4.3: Potential Extensions
* **Adding Agents**: How would you extend this setup to include more agents (e.g., a "Reviewer" agent, a "Debugger" agent)? What roles would they play?
* **Complex Problems**: How would you adapt this basic framework to solve more complex, multi-step problems that might require external tools (e.g., web search, API calls)?
* **Interactive Mode**: If you were to change `human_input_mode` to `"ALWAYS"`, describe how you would interact with the agents during the conversation.

---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* Save your Jupyter Notebook as `[YourName]_AutoGen_Basic_Agents_Assignment.ipynb`.